# Connect to Coinbase Cloud REST API and Visualise Some Stuff



## Overview

- Connect to the public coinbase.com REST API via Python
- Load data from Coinbase into Pandas
- Add some useful crypto trading indicators to the data
- Visualise Bitcoin (and/or ETH, SOL etc.) historic data in Python

Coinbase Cloud API can be found [here](https://docs.cloud.coinbase.com/exchange/reference/). 
Using the following endpoints:
- https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproducts
- https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles
- https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductstats

## Step 0 - Requirements

- Install all necessary libraries:

````
!pip install -r requirements.txt 
````


## Step 1 - Imports, Libraries and Functions

We need the following Python imports:

- **Requests:** For connecting to the REST endpoints from Coinbase.
- **Pandas** 
- **Json** 

In [ ]:

import pandas as pd
import json
import requests
from datetime import datetime, timedelta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from urllib.error import HTTPError


In [ ]:
# Choose currency that should be used to display data
FIAT_CURRENCIES = ["EUR"]#Changed the currency to EUR to test 
MY_QUOTE_CURRENCY = FIAT_CURRENCIES[0]

# I am only interested in a few currencies that I want to trade, so let's add them here:
MY_CRYPTO_CURRENCIES = ["BTC","ETH","SOL","XRP"] #added the XRP
GRANULARITIES = ['DAILY','60MIN','15MIN','1MIN'] 

CANDLE_INCREASE_COLOR = 'blue'
CANDLE_DECREASE_COLOR = 'red'
MACD_COLOR = '#A9B1F5'
MACD_SIGNAL_COLOR = '#FFD1A3'
RED_COLOR = '#D61170'
GREEN_COLOR = '#3FFA68'
#PLOTLY_TEMPLATE = 'plotly_dark'
PLOTLY_TEMPLATE = 'plotly'

In [ ]:
# Reusable method that will take an endpoint and tries to make a connection. Returns the response.
def connect(url, *args, **kwargs):  # Define a function called 'connect' that takes a URL and additional arguments (optional ones).
    try:  # Start a try block to attempt to do something that might cause an error.
        if kwargs.get('param', None) is not None:  # Check if there are parameters ('param') passed in the kwargs (optional arguments).
            params = kwargs.get('param')  # If parameters are provided, store them in the 'params' variable.
            response = requests.get(url, params)  # Make an HTTP GET request to the URL with the parameters.
        else:  # If no parameters are provided.
            response = requests.get(url)  # Make an HTTP GET request to the URL without parameters.
        response.raise_for_status()  # Check if the response status code is successful (2xx). If not, raise an HTTPError.
        print(f'HTTP connection {url} successful!')  # If no errors occur, print a success message with the URL.
        return response  # Return the response from the HTTP request.
    except HTTPError as http_err:  # Catch any HTTP-related errors.
        print(f'HTTP error occurred: {http_err}')  # Print the error message for HTTP-related errors.
    except Exception as err:  # Catch any other type of errors that might occur.
        print(f'Other error occurred: {err}')  # Print the error message for other types of errors.

# method that establishes a connection to Coinbase and will retrieve historic data for the chosen interval
def get_historic_data(start_date, end_date, interval, base_currency, quote_currency):
    # Define the valid intervals for data fetching (like DAILY, 60MIN, etc.)
    VALID_INTERVAL = {'DAILY', '60MIN', '15MIN', '5MIN', '1MIN'}

    # Check if the given interval is valid. If not, it raises an error.
    if interval not in VALID_INTERVAL:
        raise ValueError("results: interval must be one of %r." % VALID_INTERVAL)

    # Based on the selected interval, map it to the corresponding granularity (time frame in seconds)
    if interval == '1MIN':  # If the interval is '1MIN', set granularity to 60 seconds.
        granularity = '60'  
    elif interval == '15MIN':  # If the interval is '15MIN', set granularity to 900 seconds (15 minutes).
        granularity = '900'
    elif interval == '5MIN':  # If the interval is '5MIN', set granularity to 300 seconds (5 minutes).
        granularity = '300'
    elif interval == '60MIN':  # If the interval is '60MIN', set granularity to 3600 seconds (60 minutes).
        granularity = '3600'   
    else:  # If the interval is 'DAILY', set granularity to 86400 seconds (1 day).
        granularity = '86400'

    # Prepare the parameters needed for the API request (start date, end date, and granularity).
    params = {'start':start_date, 'end':end_date, 'granularity':granularity}

    # Call the 'connect' function with the constructed URL and parameters to get data from Coinbase API.
    data = json.loads(connect('https://api.exchange.coinbase.com/products/'+quote_currency+'-'+base_currency+'/candles', param = params).text)  
    # The URL is for the Coinbase API to get candlestick data. 
    # The 'quote_currency' and 'base_currency' are swapped to ensure the pair is in base-quote format like BTC-EUR instead of EUR-BTC.
    # The response is returned as JSON for easy human reading and converted to a Python list.

    # Add the quote currency (EUR) to each data entry to keep track of it.
    [x.append(quote_currency) for x in data]

    # Add the interval (DAILY, 60MIN...) to each data entry to keep track of it.
    [x.append(interval) for x in data]

    # Return the modified data with additional details about the currency and interval.
    return data

Before we start we will retrieve the time on the server and compare it to the client time:

In [ ]:
# Server time is fetched by making a GET request to Coinbase's 'time' endpoint
server_time = json.loads(connect('https://api.exchange.coinbase.com/time').text)
# The 'connect' function fetches the server time from the Coinbase API. 
# The response is then converted from JSON format to a Python dictionary using json.loads.

# Server time does not comply to iso format(yy/mm/dd), therefore slight modification of string needed
# The 'iso' key from the server's response contains the time in a format like '2025-03-07T12:45:30Z'. 
# It needs to be slightly adjusted for Python's 'fromisoformat' method, which expects a space instead of the 'T' between date and time.
server_time_now = datetime.fromisoformat(server_time['iso'].replace('T', ' ', 1)[0:19])
# Replace the 'T' with a space (just once) to match the expected ISO format, 
# then take the first 19 characters (to remove the 'Z' at the end, which signifies UTC timezone) 
# and convert it to a datetime object using 'fromisoformat'.

# Display the server time as a datetime object
print(f'Time on server: {server_time_now}')
# Print the time received from the server.

# Display the current client time
print(f'Time on client: {datetime.now()}')
# Print the current time on the client (the computer running the code).


HTTP connection https://api.exchange.coinbase.com/time successful!
Time on server: 2025-03-28 12:28:48
Time on client: 2025-03-28 12:28:46.600248


## Step 2 - Load Available Coinbase Products (Crypto Currencies)

Let's check out which currencies and trading pairs are available. You can find this information under the `products` endpoint. 

In [ ]:
# This sends a GET request to the Coinbase API to get the list of trading pairs(products).
response = connect('https://api.exchange.coinbase.com/products')

# This stores the binary content (raw response data) from the API.
response_content = response.content

# This stores the API response as a string so we can read and parse it as JSON.
response_text = response.text

# This stores metadata from the response, like id, display name, status, max slippage percentage, etc.
response_headers = response.headers

# We convert the JSON response into a Pandas DataFrame like a table with rows and columns for easier manipulation.
df_products = pd.read_json(response_text)

# Display the number of columns and rows in the DataFrame.
print("\nNumber of columns in the dataframe: %i" % (df_products.shape[1]))
print("Number of rows in the dataframe: %i\n" % (df_products.shape[0]))

# Convert the column names into a list to check them (useful for reference).
columns = list(df_products.columns)

# Sort the DataFrame by the 'id' column to organize the data.
df_products.sort_values('id', inplace=True)

# Create a copy of the DataFrame to work on without modifying the original.
df_products_formatted = df_products.copy()

# Apply formatting to the 'base_increment' column to show it with 9 decimal places.
df_products_formatted['base_increment'] = df_products_formatted['base_increment'].apply(lambda x: "{:,.9f}".format((x)))

# Filter the DataFrame to show only the rows that are related to the currencies we're interested in.
# We combine the currency pairs from MY_CRYPTO_CURRENCIES and MY_QUOTE_CURRENCY to create the matching pairs.
df_products_formatted[df_products_formatted.id.isin(list(map(lambda orig_string: orig_string+'-'+MY_QUOTE_CURRENCY, MY_CRYPTO_CURRENCIES)))][['id', 'base_currency', 'quote_currency', 'base_increment', 'max_slippage_percentage']]#added the max slippage percentage

HTTP connection https://api.exchange.coinbase.com/products successful!

Number of columns in the dataframe: 18
Number of rows in the dataframe: 679



C:\Users\hp\AppData\Local\Temp\ipykernel_13876\3215380966.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_products = pd.read_json(response_text)


,id,base_currency,quote_currency,base_increment,max_slippage_percentage
65,BTC-EUR,BTC,EUR,0.000000010,0.03
495,ETH-EUR,ETH,EUR,0.000000010,0.03
611,SOL-EUR,SOL,EUR,0.001000000,0.03
656,XRP-EUR,XRP,EUR,0.000001000,0.05


## Step 3 - Load 24h Stats

Basic stats from the last 24 hours for each of the crypto currencies that we have defined in our list above. 

In [ ]:
# Create an empty list to store data for each cryptocurrency pair.
currency_rows = []

# Loop through each cryptocurrency in MY_CRYPTO_CURRENCIES.
for currency in MY_CRYPTO_CURRENCIES:
    # Send a GET request to the Coinbase API to get the 24-hour stats for the products aka trading/pairs.
    data = json.loads(connect('https://api.exchange.coinbase.com/products/'+currency+'-'+MY_QUOTE_CURRENCY+'/stats').text)
    # Append the data for each currency pair into the currency_rows list.
    currency_rows.append(data)

# Create a DataFrame from the currency_rows list with the cryptocurrency names as the index.
df_24hstats = pd.DataFrame(currency_rows, index = MY_CRYPTO_CURRENCIES)

# Add columns for base_currency and quote_currency.
df_24hstats['base_currency'] = df_24hstats.index
df_24hstats['quote_currency'] = MY_QUOTE_CURRENCY

# Convert the columns with numeric data into the appropriate type (float).
df_24hstats['open'] = df_24hstats['open'].astype(float)
df_24hstats['high'] = df_24hstats['high'].astype(float)
df_24hstats['low'] = df_24hstats['low'].astype(float)
df_24hstats['volume'] = df_24hstats['volume'].astype(float)
df_24hstats['last'] = df_24hstats['last'].astype(float)
df_24hstats['volume_30day'] = df_24hstats['volume_30day'].astype(float)

# Calculate the performance (percentage change) for each cryptocurrency in the last 24 hours.
df_24hstats['performance'] = ((df_24hstats['last']-df_24hstats['open']) / df_24hstats['open']) * 100

# Create a copy of the DataFrame to format the values for display.
df_24hstats_formatted = df_24hstats.copy()

# Format the 'performance' column to show the percentage with two decimal places.
df_24hstats_formatted['performance'] = df_24hstats_formatted['performance'].apply(lambda x: "{:.2f}%".format((x)))

# Format the numerical columns ('open', 'high', 'low', 'last') to display them in euros with two decimal places.
df_24hstats_formatted['open'] = df_24hstats_formatted['open'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['high'] = df_24hstats_formatted['high'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['low'] = df_24hstats_formatted['low'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['last'] = df_24hstats_formatted['last'].apply(lambda x: "{:,.2f}€".format((x)))

# Print the formatted performance statistics.
print('Performance within last 24 hours in ' + MY_QUOTE_CURRENCY + ':')
print(df_24hstats_formatted)

HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/stats successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/stats successful!
HTTP connection https://api.exchange.coinbase.com/products/SOL-EUR/stats successful!
HTTP connection https://api.exchange.coinbase.com/products/XRP-EUR/stats successful!
Performance within last 24 hours in EUR:
           open        high         low        last        volume  \
BTC  80,425.42€  81,248.19€  78,718.34€  79,176.02€  3.695810e+02   
ETH   1,870.90€   1,879.00€   1,727.88€   1,759.70€  1.451825e+04   
SOL     127.57€     129.66€     120.98€     122.45€  5.509734e+04   
XRP       2.17€       2.19€       2.04€       2.08€  4.166966e+06   

     volume_30day rfq_volume_24hour rfq_volume_30day base_currency  \
BTC  1.417207e+04          6.940315       508.567050           BTC   
ETH  3.335711e+05        218.798321     13198.052986           ETH   
SOL  1.808529e+06       3897.934672    298948.843959           

## Step 4 - Load Historic Data

Download data using four different time slices and periods:
- Daily data of the last 90 days
- Hourly data of the last 300 hours
- 15-minute data of the last 75 hours
- 1-minute data of the last 2 hours

All this data is stored in a pandas dataframe called df_history.

CHANGE THIS TO RETREIVE MORE DATA!

In [ ]:
# Initialize an empty list to store historical data for each cryptocurrency
currency_history_rows = []

# Loop through each cryptocurrency in MY_CRYPTO_CURRENCIES
for currency in MY_CRYPTO_CURRENCIES:
    # Set the end date to the current server time in ISO format
    end_date = server_time_now.isoformat()

    # Collect data for different time intervals
    # 1-minute data (past 2 hours)
    start_date = (server_time_now - timedelta(hours=2)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date, end_date, '1MIN', MY_QUOTE_CURRENCY, currency))
    
    # 15-minute data (past 75 hours)
    start_date = (server_time_now - timedelta(hours=75)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date, end_date, '15MIN', MY_QUOTE_CURRENCY, currency))
    
    # 60-minute data (past 300 hours or 12.5 days)
    start_date = (server_time_now - timedelta(hours=300)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date, end_date, '60MIN', MY_QUOTE_CURRENCY, currency))
    
    # Daily data (past 90 days)
    start_date = (server_time_now - timedelta(days=90)).isoformat()
    currency_history_rows.extend(get_historic_data(start_date, end_date, 'DAILY', MY_QUOTE_CURRENCY, currency))

# Create a DataFrame from the collected historical data
df_history = pd.DataFrame(currency_history_rows)

# Rename the columns based on the Coinbase API documentation
df_history.columns = ['time', 'low', 'high', 'open', 'close', 'volume', 'base_currency', 'granularity']

# Add additional columns for better readability
df_history['quote_currency'] = MY_QUOTE_CURRENCY  # Set the quote currency (EUR)
df_history['date'] = pd.to_datetime(df_history['time'], unit='s')  # Convert time (Unix) to datetime
df_history['year'] = pd.DatetimeIndex(df_history['date']).year  # Extract year from the date
df_history['month'] = pd.DatetimeIndex(df_history['date']).month  # Extract month from the date
df_history['day'] = pd.DatetimeIndex(df_history['date']).day  # Extract day from the date
df_history['hour'] = pd.DatetimeIndex(df_history['date']).hour  # Extract hour from the date
df_history['minute'] = pd.DatetimeIndex(df_history['date']).minute  # Extract minute from the date


HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/BTC-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/ETH-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/SOL-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/SOL-EUR/candles successful!
HTTP connection https://api.exchange.coinbase.com/products/SOL-EUR/candles successful!
HTTP connection https://api.exchange.coinba

In [ ]:
print(f'Daily data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'DAILY\'')

Daily data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
720,1743120000,78718.3400,81029.4000,80780.1500,79176.0200,2.130973e+02,BTC,DAILY,EUR,2025-03-28,2025,3,28,0,0
721,1743033600,80654.5900,80989.3400,80969.7500,80751.7300,9.353113e+00,BTC,DAILY,EUR,2025-03-27,2025,3,27,0,0
722,1742947200,79838.5100,81878.0000,81050.8900,80976.0400,3.307513e+02,BTC,DAILY,EUR,2025-03-26,2025,3,26,0,0
723,1742860800,79916.9500,82067.9000,81014.1100,81040.5000,3.394556e+02,BTC,DAILY,EUR,2025-03-25,2025,3,25,0,0
724,1742774400,79010.5500,82091.4300,79441.3400,81016.5800,4.394452e+02,BTC,DAILY,EUR,2025-03-24,2025,3,24,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3234,1735776000,2.2507,2.3738,2.2507,2.3398,4.042435e+06,XRP,DAILY,EUR,2025-01-02,2025,1,2,0,0
3235,1735689600,2.0055,2.2696,2.0068,2.2493,3.163453e+06,XRP,DAILY,EUR,2025-01-01,2025,1,1,0,0
3236,1735603200,1.9325,2.0667,1.9744,2.0069,2.327001e+06,XRP,DAILY,EUR,2024-12-31,2024,12,31,0,0
3237,1735516800,1.9178,2.0592,2.0031,1.9738,3.428513e+06,XRP,DAILY,EUR,2024-12-30,2024,12,30,0,0


In [ ]:
print(f'60 minute data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'60MIN\'').tail(5)

60 minute data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
3144,1742101200,2.1906,2.2012,2.1913,2.1989,11288.207754,XRP,60MIN,EUR,2025-03-16 05:00:00,2025,3,16,5,0
3145,1742097600,2.1908,2.2015,2.1917,2.1916,28689.709000,XRP,60MIN,EUR,2025-03-16 04:00:00,2025,3,16,4,0
3146,1742094000,2.1774,2.1930,2.1796,2.1911,349847.737339,XRP,60MIN,EUR,2025-03-16 03:00:00,2025,3,16,3,0
3147,1742090400,2.1737,2.1917,2.1858,2.1805,277468.525290,XRP,60MIN,EUR,2025-03-16 02:00:00,2025,3,16,2,0
3148,1742086800,2.1874,2.2183,2.2123,2.1874,41310.180918,XRP,60MIN,EUR,2025-03-16 01:00:00,2025,3,16,1,0


In [ ]:
print(f'15 minute data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'15MIN\'').tail(5)

15 minute data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
2844,1742898600,2.2537,2.2624,2.2549,2.2615,12271.854680,XRP,15MIN,EUR,2025-03-25 10:30:00,2025,3,25,10,30
2845,1742897700,2.2508,2.2555,2.2552,2.2547,5710.197462,XRP,15MIN,EUR,2025-03-25 10:15:00,2025,3,25,10,15
2846,1742896800,2.2532,2.2609,2.2609,2.2542,10789.029469,XRP,15MIN,EUR,2025-03-25 10:00:00,2025,3,25,10,0
2847,1742895900,2.2563,2.2620,2.2567,2.2615,7463.646431,XRP,15MIN,EUR,2025-03-25 09:45:00,2025,3,25,9,45
2848,1742895000,2.2536,2.2621,2.2537,2.2583,20558.852274,XRP,15MIN,EUR,2025-03-25 09:30:00,2025,3,25,9,30


In [ ]:
print(f'1 minute data included in df_history (in {MY_QUOTE_CURRENCY}):')
df_history.query('granularity == \'1MIN\'').tail(5)

1 minute data included in df_history (in EUR):


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,year,month,day,hour,minute
2544,1743157980,2.0670,2.0679,2.0678,2.0674,128.426476,XRP,1MIN,EUR,2025-03-28 10:33:00,2025,3,28,10,33
2545,1743157920,2.0668,2.0682,2.0668,2.0672,230.114409,XRP,1MIN,EUR,2025-03-28 10:32:00,2025,3,28,10,32
2546,1743157860,2.0673,2.0687,2.0673,2.0681,3383.415215,XRP,1MIN,EUR,2025-03-28 10:31:00,2025,3,28,10,31
2547,1743157800,2.0653,2.0671,2.0653,2.0671,1051.995999,XRP,1MIN,EUR,2025-03-28 10:30:00,2025,3,28,10,30
2548,1743157740,2.0654,2.0664,2.0654,2.0657,1430.840970,XRP,1MIN,EUR,2025-03-28 10:29:00,2025,3,28,10,29


## Step 5 - Insights

Let's add a few more parameters to the basic data (available in df_history):

- **SMA3 and SMA7 and SMA12** (Simple Moving Average over last 3 and 7 time slices): This is a price based, lagging (or reactive) indicator that displays the average price of a security over a certain period of time. Moving Averages smooth out the “noise” when interpreting charts. Noise is made up of fluctuations of both price and volume. Because a Moving Average is a lagging indicator and reacts to events that have already happened, it is not used as a predictive indicator but rather an interpretive one. Simple Moving Average is an unweighted Moving Average. This means that each period in the data set has equal importance and is weighted equally. 
- **EMA12 and EMA26** (Exponential Moving Average over last 12 and 26 time slices): The EMA has much less lag than the SMA because it puts a greater importance on more recent prices. It therefore turns quicker than the SMA. You can choose different time slices, but 12 and 26 are commonly used durations.
- **MACD** (Moving Average Convergence/Divergence): A great indicator to identify the overall trend of any security (e.g. crypto currencies). It takes the difference between the short term EMA and the long term EMA. A positive MACD value is an indicator for a positive market trend. A negative MACD value is an indicator for a negative market trend. 
- **MACD Signal**: The MACD Signal line is the EMA over a certain set of time slices of the MACD line. Typically, the value is set to 9 time slices. 
- **MACD Histogram**: It is the difference between the MACD line and the MACD Signal line. A bullish crossover occurs when the MACD line crosses above the MACD Signal line. A bearish crossover occurs if the MACD line crosses below the MACD Signal line. 
- **Open to close performance**: For each time slice, the difference between the closing price and the opening price of that particular period is given as a percentage.
- **High low difference**: The deviation within the current period between the highest and lowest price as a percentage. This shows the volatility within one time slice.
- **Current high versus previous close absolute value**: The absolute value of the difference between the high of the current period and the close value of the previous period (needed for ATR calculation)
- **Current low versus previous close absolute value**: The absolute value of the difference between the low of the current period and the close value of the previous period (needed for ATR calculation)
- **True Range**: The max value of the previous three values (needed for ATR calculation).
- **Average True Range (ATR)**:  This is a is a market volatility indicator used in technical analysis. It takes the moving average of the True Range (typically over a 14 day period). It is not (!) an indicator for price direction. 
- **Absolute performance of last 3 periods**: This indicator provides the performance over the last three time slices as an absolute value.
- **Performance of last 3 periods in percentage**: This indicator provides the performance over the last three time slices as a relative value.
- **Bull or Bear market**: It simply writes "Bull" in case the MACD Histogram is positive and it writes "Bear" in case the MACD Histogram value is negative. 
- **Market trend continued**: To identify a shift from Bull to Bear market and vice versa, this column holds "True" in case the trend from the previous time slice continues and "False" in case a trend change has occurred. 

The enhanced data is stored in a pandas dataframe called df_history_enhanced. 

Learn more about the different trading indicators like SMA, EMA, MACD and ATR, take a look [here](https://www.tradingview.com)

In [ ]:
# Set the period for calculating Average True Range (ATR)
AVERAGE_TRUE_RANGE_PERIODS = 14

# Create an empty list to store the enhanced data
currency_history_rows_enhanced = []

# Loop through each currency in MY_CRYPTO_CURRENCIES
for currency in MY_CRYPTO_CURRENCIES:
    # Loop through each granularity (time interval) in GRANULARITIES
    for granularity in GRANULARITIES:
        # Filter the data for the current currency and granularity
        df_history_currency = df_history.query('granularity == @granularity & base_currency == @currency').copy()
        
        # Sort the data by date from oldest to newest (needed for calculations)
        df_history_currency = df_history_currency.sort_values(['date'], ascending=True)
        
        # Calculate Simple Moving Averages (SMA) for 3 and 7 periods
        df_history_currency['SMA3'] = df_history_currency['close'].rolling(window=3).mean()
        df_history_currency['SMA7'] = df_history_currency['close'].rolling(window=7).mean()
        
        # Calculate Exponential Moving Averages (EMA) for 12 and 26 periods
        df_history_currency['EMA12'] = df_history_currency['close'].ewm(span=12, adjust=False).mean()
        df_history_currency['EMA26'] = df_history_currency['close'].ewm(span=26, adjust=False).mean()
        
        # Calculate MACD (Moving Average Convergence Divergence)
        df_history_currency['MACD'] = df_history_currency['EMA12'] - df_history_currency['EMA26']
        
        # Calculate MACD signal line (9-period EMA of MACD)
        df_history_currency['MACD_signal'] = df_history_currency['MACD'].ewm(span=9, adjust=False).mean()
        
        # Calculate MACD histogram (difference between MACD and signal line)
        df_history_currency['macd_histogram'] = df_history_currency['MACD'] - df_history_currency['MACD_signal']
        
        # Calculate open-to-close performance
        df_history_currency['open_to_close_perf'] = ((df_history_currency['close'] - df_history_currency['open']) / df_history_currency['open'])
        
        # Calculate the difference between high and low prices
        df_history_currency['high_low_diff'] = df_history_currency['high'] - df_history_currency['low']
        
        # Calculate the absolute difference between high and previous close
        df_history_currency['high_prev_close_abs'] = np.abs(df_history_currency['high'] - df_history_currency['close'].shift())
        
        # Calculate the absolute difference between low and previous close
        df_history_currency['low_prev_close_abs'] = np.abs(df_history_currency['low'] - df_history_currency['close'].shift())
        
        # Calculate True Range (max of high-low, high-prev close, low-prev close)
        df_history_currency['true_range'] = df_history_currency[['high_low_diff', 'high_prev_close_abs', 'low_prev_close_abs']].max(axis=1)
        
        # Calculate Average True Range using an Exponential Moving Average (EMA)
        df_history_currency['average_true_range'] = df_history_currency['true_range'].ewm(alpha=1/AVERAGE_TRUE_RANGE_PERIODS, min_periods=AVERAGE_TRUE_RANGE_PERIODS, adjust=False).mean()
        
        # Calculate the performance of high-low difference relative to high price
        df_history_currency['high_low_perf'] = (df_history_currency['high'] - df_history_currency['low']) / df_history_currency['high']
        
        # Calculate the open price performance over the last 3 periods (absolute and percentage change)
        df_history_currency['open_perf_last_3_period_abs'] = df_history_currency['open'].rolling(window=4).apply(lambda x: x.iloc[1] - x.iloc[0])
        df_history_currency['open_perf_last_3_period_per'] = df_history_currency['open'].rolling(window=4).apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0])
        
        # Identify whether the market is bullish or bearish based on MACD histogram
        df_history_currency['bull_bear'] = np.where(df_history_currency['macd_histogram'] < 0, 'Bear', 'Bull')
        
        # Append the enhanced data for the current currency and granularity to the list
        currency_history_rows_enhanced.append(df_history_currency)

# Combine all the enhanced data into a single DataFrame
df_history_enhanced = pd.concat(currency_history_rows_enhanced, ignore_index=True)

# Sort the data by base currency, granularity, and date for trend analysis
df_history_enhanced = df_history_enhanced.sort_values(['base_currency', 'granularity', 'date'], ascending=True)

# Identify if the market trend continued (bullish/bearish) from the previous period
df_history_enhanced['market_trend_continued'] = df_history_enhanced.bull_bear.eq(df_history_enhanced.bull_bear.shift()) & \
                                                df_history_enhanced.base_currency.eq(df_history_enhanced.base_currency.shift()) & \
                                                df_history_enhanced.granularity.eq(df_history_enhanced.granularity.shift())

# Show the first 10 rows of the enhanced DataFrame
df_history_enhanced.head(10)


,time,low,high,open,close,volume,base_currency,granularity,quote_currency,date,...,high_low_diff,high_prev_close_abs,low_prev_close_abs,true_range,average_true_range,high_low_perf,open_perf_last_3_period_abs,open_perf_last_3_period_per,bull_bear,market_trend_continued
390,1742895000,80455.76,80878.60,80472.54,80699.61,10.008546,BTC,15MIN,EUR,2025-03-25 09:30:00,...,422.84,NaN,NaN,422.84,NaN,0.005228,NaN,NaN,Bull,False
391,1742895900,80628.03,80885.74,80665.02,80783.80,3.948973,BTC,15MIN,EUR,2025-03-25 09:45:00,...,257.71,186.13,71.58,257.71,NaN,0.003186,NaN,NaN,Bull,True
392,1742896800,80526.61,80832.83,80778.44,80551.92,4.460579,BTC,15MIN,EUR,2025-03-25 10:00:00,...,306.22,49.03,257.19,306.22,NaN,0.003788,NaN,NaN,Bear,False
393,1742897700,80416.00,80619.11,80552.10,80545.14,2.666942,BTC,15MIN,EUR,2025-03-25 10:15:00,...,203.11,67.19,135.92,203.11,NaN,0.002519,192.48,0.002392,Bear,True
394,1742898600,80488.18,80683.30,80542.21,80668.48,3.701614,BTC,15MIN,EUR,2025-03-25 10:30:00,...,195.12,138.16,56.96,195.12,NaN,0.002418,113.42,0.001406,Bear,True
395,1742899500,80535.76,80678.88,80665.65,80563.91,4.477297,BTC,15MIN,EUR,2025-03-25 10:45:00,...,143.12,10.40,132.72,143.12,NaN,0.001774,-226.34,-0.002802,Bear,True
396,1742900400,80523.42,80715.58,80582.98,80630.14,2.089202,BTC,15MIN,EUR,2025-03-25 11:00:00,...,192.16,151.67,40.49,192.16,NaN,0.002381,-9.89,-0.000123,Bear,True
397,1742901300,80636.09,80804.63,80643.66,80767.58,1.525702,BTC,15MIN,EUR,2025-03-25 11:15:00,...,168.54,174.49,5.95,174.49,NaN,0.002086,123.44,0.001533,Bull,False
398,1742902200,80665.96,80884.00,80792.37,80856.23,3.815750,BTC,15MIN,EUR,2025-03-25 11:30:00,...,218.04,116.42,101.62,218.04,NaN,0.002696,-82.67,-0.001025,Bull,True
399,1742903100,80773.53,80867.43,80864.02,80809.60,3.510041,BTC,15MIN,EUR,2025-03-25 11:45:00,...,93.90,11.20,82.70,93.90,NaN,0.001161,60.68,0.000753,Bull,True


## Step 6 - Print Out Selected Information

Now, we can leverage the data which is available in df_history_enhanced to compile some basic parameters that we might want to use in a personal dashboard down the line.

In [ ]:
# Loop through all crypto currencies to gather and process additional market data
additional_info_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True)  # Sort data from oldest to newest
    row_data = [
        currency,
        # Current value of the cryptocurrency
        float(df_24hstats.query('base_currency == @currency')['last'].to_string(index=False)),
        # Current market trend for 1 minute and previous 1 minute trend
        df_history_enhanced.query('granularity == \'1MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False),
        df_history_enhanced.query('granularity == \'1MIN\' and base_currency == @currency').iloc[-2]['bull_bear'],
        # Market trend for 15 minutes and previous 15 minutes trend
        df_history_enhanced.query('granularity == \'15MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False),
        df_history_enhanced.query('granularity == \'15MIN\' and base_currency == @currency').iloc[-2]['bull_bear'],
        # Market trend for 60 minutes and previous 60 minutes trend
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(1)['bull_bear'].to_string(index=False),
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').iloc[-2]['bull_bear'],
        # Lowest price in the last 24 hours, 30 days, and 90 days
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(24)['low'].min(),
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(30)['low'].min(),
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['low'].min(),
        # Highest price in the last 24 hours, 30 days, and 90 days
        df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency').tail(24)['high'].max(),
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(30)['high'].max(),
        df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['high'].max(),
        # Max high/low difference over the last 90 days and 12 days
        round(df_history_enhanced.query('granularity == \'DAILY\' and base_currency == @currency').tail(90)['high_low_perf'].max(), 3),
        round(df_history_enhanced.query('granularity == \'60MIN\' and base_currency == @currency')['high_low_perf'].max(), 3)
    ]
    # Add the gathered row data to the list
    additional_info_rows.append(row_data)

# Convert the data into a DataFrame and format numerical values
df_additional_info = pd.DataFrame(additional_info_rows, columns=['base_currency', 'current_val', '1min_trend', '1min_prev_trend', 
                                                                 '15min_trend', '15min_prev_trend', '60min_trend', '60min_prev_trend', 
                                                                 '24h_low', '30day_low', '90day_low', '24h_high', '30day_high', '90day_high', 
                                                                 '90day_max_high_low_span', '12day_max_high_low_span'])

# Formatting the values for display
df_additional_info['current_val'] = df_additional_info['current_val'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['24h_low'] = df_additional_info['24h_low'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['30day_low'] = df_additional_info['30day_low'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['90day_low'] = df_additional_info['90day_low'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['24h_high'] = df_additional_info['24h_high'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['30day_high'] = df_additional_info['30day_high'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['90day_high'] = df_additional_info['90day_high'].apply(lambda x: "{:,.2f}$".format((x)))
df_additional_info['90day_max_high_low_span'] = (df_additional_info['90day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))
df_additional_info['12day_max_high_low_span'] = (df_additional_info['12day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))

# Transpose the DataFrame for better viewing
df_additional_info.set_index('base_currency').transpose()

base_currency,BTC,ETH,SOL,XRP
current_val,"79,176.02$","1,759.70$",122.45$,2.08$
1min_trend,Bull,Bull,Bull,Bull
1min_prev_trend,Bull,Bull,Bull,Bull
15min_trend,Bull,Bull,Bull,Bull
15min_prev_trend,Bull,Bull,Bull,Bull
60min_trend,Bear,Bear,Bear,Bear
60min_prev_trend,Bear,Bear,Bear,Bear
24h_low,"78,718.34$","1,727.88$",120.98$,2.04$
30day_low,"70,570.00$","1,616.86$",103.31$,1.75$
90day_low,"70,570.00$","1,616.86$",103.31$,1.73$


## Step 7 - Visualise Data

For visualisation - use plotly.

### Simple Visualisation

First, let's create a simple OHLC (Open, High, Low, Close) candle stick chart for each of our currencies.

In [ ]:
#!pip3 install plotly
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)  # Initialize Plotly in offline mode for notebook compatibility
import plotly.graph_objs as go  # Import Plotly graphing objects for creating visualizations

df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True)  # Sort data by date (oldest to newest) for visualization

granularity = '15MIN'  # Set the granularity (time interval) for the chart

# Loop over each cryptocurrency in MY_CRYPTO_CURRENCIES to generate a chart for each
for currency in MY_CRYPTO_CURRENCIES:
    # Filter data for the specific granularity and currency
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'')
    
    fig = go.Figure()  # Create a new figure for the chart

    # Add a Candlestick trace (OHLC chart) for the currency
    fig.add_trace(go.Candlestick(
        x=df_history_for_chart['date'],  # Set the x-axis to be the date
        open=df_history_for_chart['open'],  # Set the open prices
        high=df_history_for_chart['high'],  # Set the high prices
        low=df_history_for_chart['low'],  # Set the low prices
        close=df_history_for_chart['close'],  # Set the close prices
        name='OHLC'  # Name of the trace
    ))

    # Add the Average True Range (ATR) line on a secondary y-axis
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],  # Set the x-axis to be the date
        y=df_history_for_chart['average_true_range'],  # Set the y-values to the ATR
        line=dict(color=MACD_COLOR),  # Set the line color for ATR (should be predefined)
        mode='lines',  # Display the ATR as a line
        name="ATR",  # Name of the ATR trace
        yaxis="y2"  # Attach ATR to the secondary y-axis
    ))

    # Customize candlestick colors based on price movement
    cs = fig.data[0]  # Access the candlestick trace
    cs.increasing.fillcolor = CANDLE_INCREASE_COLOR  # Set color for increasing candles
    cs.increasing.line.color = CANDLE_INCREASE_COLOR  # Set border color for increasing candles
    cs.decreasing.fillcolor = CANDLE_DECREASE_COLOR  # Set color for decreasing candles
    cs.decreasing.line.color = CANDLE_DECREASE_COLOR  # Set border color for decreasing candles

    # Update layout settings for the chart
    fig.update_layout(
        yaxis=dict(
            title=MY_QUOTE_CURRENCY,  # Set the title for the primary y-axis (currency values)
            side="right",  # Place the y-axis on the right side
        ),
        yaxis2=dict(
            title="Average True Range (Volatility)",  # Title for the secondary y-axis (ATR)
            anchor="free",  # Set to free to avoid overlaying with the primary y-axis
            overlaying="y",  # Overlay this y-axis over the primary one
            side="left",  # Place the secondary y-axis on the left
        ),
        yaxis2_showgrid=False,  # Disable grid for the secondary y-axis
        yaxis2_zeroline=False,  # Disable the zero line for the secondary y-axis
        xaxis_rangeslider_visible=False,  # Hide the range slider below the x-axis
        title=f'OHLC Chart for {currency} with time interval {granularity}',  # Set the chart title
        hovermode='x unified',  # Show all data for a specific x-axis point when hovering
        xaxis=dict(
            rangeslider=dict(visible=True),  # Make the range slider visible for zooming
            type="date"  # Set the x-axis to display dates
        ),
        template=PLOTLY_TEMPLATE,  # Apply a custom Plotly template (should be predefined)
    )

    # Display the chart
    fig.show()

### Advanced Visualization

Now that we have drawn a basic OHLC chart, let's get a bit more fancy. We will now draw the chart again. But this time we will add the following information:
- MACD line
- MACD signal line
- Colored area to show if we are currently in a bull market (green) or bear market (red) in each given period

Construct the data needed to color different periods for the market trends (bull versus bear). Create another pandas dataframe df_history_market_trend_intervals.

In [ ]:
# Initialize an empty list to store market trend intervals for all cryptocurrencies
market_trend_interval_rows = []

# Loop through each cryptocurrency in the MY_CRYPTO_CURRENCIES list
for currency in MY_CRYPTO_CURRENCIES:
    # Loop through each granularity type in the GRANULARITIES list
    for granularity in GRANULARITIES:
        # Filter data for the specific currency, where market_trend_continued is False and granularity matches
        df_history_market_trend_intervals = df_history_enhanced.query(
            'base_currency == @currency and market_trend_continued == False and granularity == @granularity'
        ).copy()

        # Create a new column 'next_period_date' by shifting the 'date' column by one period forward
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals.date.shift(-1)
        
        # Fill any NaN values in the 'next_period_date' column with the current date and time
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals['next_period_date']
        df_history_market_trend_intervals.next_period_date = df_history_market_trend_intervals.next_period_date.fillna(datetime.now())
        
        # Create a new column 'color' to assign a color based on the 'bull_bear' value (Bull -> GREEN, Bear -> RED)
        df_history_market_trend_intervals['color'] = df_history_market_trend_intervals['bull_bear'].apply(
            lambda x: GREEN_COLOR if x == 'Bull' else RED_COLOR
        )
        
        # Select and rename relevant columns for market trend intervals: base_currency, granularity, bull_bear, color, date, and next_period_date
        df_history_market_trend_intervals = df_history_market_trend_intervals[[
            'base_currency', 'granularity', 'bull_bear', 'color', 'date', 'next_period_date'
        ]].rename(columns={"date": "start_date", "next_period_date": "finish_date"})
        
        # Append the processed DataFrame to the list of market trend interval rows
        market_trend_interval_rows.append(df_history_market_trend_intervals)

# Concatenate all the market trend interval rows into a single DataFrame
df_history_market_trend_intervals = pd.concat(market_trend_interval_rows, ignore_index=True)

In [ ]:
# CHANGE HERE: 1MIN, 15MIN, 60MIN, DAILY
granularity = '60MIN'  # Set the desired granularity for the chart (options: '1MIN', '15MIN', '60MIN', 'DAILY')

# Loop through each cryptocurrency in the MY_CRYPTO_CURRENCIES list
for currency in MY_CRYPTO_CURRENCIES:
    # Filter data for the specific currency and granularity from the df_history_enhanced DataFrame
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'').copy()
    
    # Filter data for the specific currency and granularity from the df_history_market_trend_intervals DataFrame
    df_history_market_trend_for_chart = df_history_market_trend_intervals.query(f'granularity == \'{granularity}\' and base_currency == \'{currency}\'').copy()
    
    # Get the minimum and maximum values of the MACD for the chart's y-axis range
    macd_min = df_history_for_chart['MACD'].min()
    macd_max = df_history_for_chart['MACD'].max()
    
    # Create a figure with secondary y-axis support for multiple traces
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add OHLC (Open, High, Low, Close) data to the chart, including color customization for increasing and decreasing candles
    fig.add_trace(go.Ohlc(
        x=df_history_for_chart['date'],
        open=df_history_for_chart['open'],
        high=df_history_for_chart['high'],
        low=df_history_for_chart['low'],
        close=df_history_for_chart['close'],
        increasing_line_color=CANDLE_INCREASE_COLOR,  # Set the color for increasing candles
        decreasing_line_color=CANDLE_DECREASE_COLOR,  # Set the color for decreasing candles
        name='OHLC'), secondary_y=True)
    
    # Add MACD (Moving Average Convergence Divergence) data to the chart
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD'],
        name='MACD',
        line=dict(color=MACD_COLOR),  # Set the color for the MACD line
        mode='lines'), secondary_y=False)
    
    # Add MACD Signal data to the chart (the signal line for MACD)
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD_signal'],
        name='MACD Signal',
        line=dict(color=MACD_SIGNAL_COLOR),  # Set the color for the MACD Signal line
        mode='lines'), secondary_y=False)
    
    # Loop through the market trend intervals for the current currency to add vertical rectangles indicating trend periods
    for i in range(df_history_market_trend_for_chart['base_currency'].count()):
        fig.add_vrect(
            x0=df_history_market_trend_for_chart.iloc[i]['start_date'],  # Start of the trend interval
            x1=df_history_market_trend_for_chart.iloc[i]['finish_date'],  # End of the trend interval
            col=1,
            # Optional: You can set the fill color for the rectangle, currently commented out
            line=dict(color=df_history_market_trend_for_chart.iloc[i]['color']),  # Set the line color of the rectangle (based on the trend)
            opacity=1.0,
            line_width=1
        )
    
    # Update the layout of the figure to set various chart properties
    fig.update_layout(
        title=f'OHLC Chart for {currency} with time interval {granularity}',  # Set chart title
        hovermode='x unified',  # Set hover mode to show data across the x-axis
        legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.94),  # Position the legend
        template=PLOTLY_TEMPLATE,  # Set the plot template (style)
        yaxis_range=[macd_min, macd_max],  # Set the y-axis range based on the MACD data
        yaxis1_showgrid=False,  # Hide grid lines for the primary y-axis
        yaxis1_showticklabels=False,  # Hide tick labels for the primary y-axis
        yaxis1_zeroline=False,  # Hide the zero line for the primary y-axis
        margin=dict(
            l=50,  # Left margin
            r=50,  # Right margin
            b=100,  # Bottom margin
            t=100,  # Top margin
            pad=4  # Padding for the margins
        ),
        xaxis=dict(
            rangeselector=dict(
                buttons=[dict(step="all")]  # Set the range selector to show all data
            ),
            rangeslider=dict(visible=True),  # Add a range slider below the x-axis
            type="date"  # Set the x-axis to represent dates
        )
    )
    
    # Set the x-axis tick format depending on the granularity (time interval)
    if granularity == 'DAILY':
        fig.update_layout(xaxis_tickformat='%d %b (%a)')  # Format the date as day/month (week day)
    else:
        fig.update_layout(xaxis_tickformat='%d %b %H:%M')  # Format the date as day/month hour:minute
    
    # Display the figure
    fig.show()


# BOOM!

TESTING AROUND 

In [ ]:
import pandas as pd  # For handling datasets and manipulating tables
import requests  # To make API requests and fetch data from the web
import numpy as np  # For numerical operations and calculations
import plotly.graph_objects as go  # For creating visualizations
from plotly.subplots import make_subplots  # To create multiple plots in one figure

# Function to fetch Coinbase historical data
def fetch_coinbase_data(product_id='BTC-USD', granularity=86400):
    """Fetches historical cryptocurrency data from Coinbase API."""
    url = f'https://api.exchange.coinbase.com/products/{product_id}/candles?granularity={granularity}'  # Construct API endpoint
    response = requests.get(url)  # Send request to Coinbase API
    data = response.json()  # Convert response to JSON format
    
    # Create a DataFrame with labeled columns
    df = pd.DataFrame(data, columns=['timestamp', 'low', 'high', 'open', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')  # Convert timestamp to readable date format
    df = df.sort_values('timestamp')  # Ensure data is in chronological order
    return df  # Return formatted dataset

# Function to calculate ATR (Average True Range)
def calculate_atr(df, period=14):
    """Calculates Average True Range (ATR) to measure market volatility."""
    df['high-low'] = df['high'] - df['low']  # High minus low price
    df['high-prev_close'] = abs(df['high'] - df['close'].shift(1))  # High minus previous close
    df['low-prev_close'] = abs(df['low'] - df['close'].shift(1))  # Low minus previous close
    df['TR'] = df[['high-low', 'high-prev_close', 'low-prev_close']].max(axis=1)  # True range calculation
    df['ATR'] = df['TR'].rolling(window=period).mean()  # Moving average of true range
    return df

# Function to calculate EMA (Exponential Moving Average)
def calculate_ema(df, period=50):
    """Calculates the Exponential Moving Average (EMA) for trend analysis."""
    df[f'EMA_{period}'] = df['close'].ewm(span=period, adjust=False).mean()  # EMA formula
    return df

# Function to calculate RSI (Relative Strength Index)
def calculate_rsi(df, period=14):
    """Calculates the RSI to identify overbought or oversold conditions."""
    delta = df['close'].diff(1)  # Price difference between days
    gain = np.where(delta > 0, delta, 0)  # Keep only gains
    loss = np.where(delta < 0, -delta, 0)  # Keep only losses
    avg_gain = pd.Series(gain).rolling(window=period, min_periods=1).mean()  # Average gain over time
    avg_loss = pd.Series(loss).rolling(window=period, min_periods=1).mean()  # Average loss over time
    rs = avg_gain / avg_loss  # Relative strength calculation
    df['RSI'] = 100 - (100 / (1 + rs))  # RSI formula
    return df

# Function to detect whale activity with improved visualization
def detect_whale_activity(df, volume_threshold=2.0):
    """Detects whale activity based on sudden large volume spikes and flags them in the chart."""
    df['Avg_Volume'] = df['volume'].rolling(window=20).mean()  # Calculate 20-day average volume
    df['Whale_Alert'] = df['volume'] > (df['Avg_Volume'] * volume_threshold)  # Flag whale trades
    return df

# Fetch data and apply calculations
df = fetch_coinbase_data()  # Get price data
df = calculate_atr(df)  # Compute ATR

df = calculate_ema(df, period=50)  # Compute EMA
df = calculate_rsi(df, period=14)  # Compute RSI
df = detect_whale_activity(df)  # Detect whale trades

# Plot results
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

# Price chart with EMA
fig.add_trace(go.Candlestick(x=df['timestamp'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='Price'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['EMA_50'], mode='lines', name='EMA 50', line=dict(color='orange')), row=1, col=1)

# Highlight Whale Activity on Price Chart
whale_data = df[df['Whale_Alert']]  # Filter whale trades
fig.add_trace(go.Scatter(x=whale_data['timestamp'], y=whale_data['close'], mode='markers', marker=dict(size=10, color='blue'), name='Whale Alert'), row=1, col=1)

# ATR plot
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['ATR'], mode='lines', name='ATR', line=dict(color='pink')), row=2, col=1)

# RSI plot
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['RSI'], mode='lines', name='RSI', line=dict(color='purple')), row=3, col=1)

fig.update_layout(title='Coinbase BTC Analysis with ATR, EMA & RSI (Whale Alerts)', xaxis_title='Date', template='plotly_dark', height=900)
fig.show()

# Display detected whale activity
df[df['Whale_Alert']][['timestamp', 'volume', 'Avg_Volume', 'close']]  # Show whale trades in table format


,timestamp,volume,Avg_Volume,close
317,2024-05-15,21821.388587,10791.169471,66244.69
290,2024-06-11,17354.689155,8625.015086,67316.53
277,2024-06-24,21418.045946,9227.657807,60263.00
266,2024-07-05,28052.415017,10976.741601,56639.43
235,2024-08-05,59125.789267,12392.235541,54029.12
150,2024-10-29,20616.911870,9481.884665,72724.35
142,2024-11-06,45398.118374,11720.156845,75646.55
137,2024-11-11,35560.118667,14835.483658,88770.73
136,2024-11-12,40111.809246,16390.015550,88035.43
135,2024-11-13,42318.690917,18005.140181,90493.82
